### Extracting features - Finance analysis

### Features Extracted:
    - Numero de Itens do projeto.
    - Total Captado.
    - Total Comprovado.
    - Total Aprovado.
    - Número de Comprovantes.
    - % de fornecedores entre os mais comuns.
    - % de itens cujo valor é acima da mediana (no municipio / estado).
    - % itens do projeto entre os X mais comuns no segmento.
    - Número de projetos já enviados pelo "Proponente".
    - Como a planilha evolui no tempo (número de modificações por rubrica).
    
Gerar features de um segmento "ESPECÍFICO" e "GLOBAIS".

### Importing data

In [12]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats


from salicml.utils.dates import Dates
from salicml.utils.read_csv import read_csv


PROJECT_ROOT = os.path.abspath(os.path.join(os.pardir, os.pardir))
DATA_FOLDER = os.path.join(PROJECT_ROOT, 'data', 'raw')

#### PLANILHA ORCAMENTARIA
QUERY:


In [13]:
PlanilhaOrcamentaria = 'planilha_orcamentaria.csv'
planilhaOrcamentaria = read_csv(PlanilhaOrcamentaria)

planilhaOrcamentaria.head()

,idPronac,PRONAC,idPlanilhaAprovacao,Item,idPlanilhaItens,Unidade,QtDias,QtItem,nrOcorrencia,VlUnitarioSolicitado,...,UfItem,MunicipioItem,Etapa,idArea,Area,idSegmento,Segmento,idProduto,Produto,DataProjeto
0,114166,93004,50109,Montagem e desmontagem,88,Serviço,2,1.0,2,140.00,...,PB,Sumé,2 - Produção / Execução,5,Patrimônio Cultural,51,História,62,Exposição de Artes,2009-06-26 09:46:54
1,123461,103228,239572,Locação de Piano,3040,Serviço,12,1.0,6,1500.00,...,RJ,Rio de Janeiro,2 - Produção / Execução,3,Música,33,Música Instrumental,51,Apresentação Musical,2010-05-06 10:49:13
2,123461,103228,239599,Programa,2634,Unidade,6,400.0,6,2.00,...,RJ,Rio de Janeiro,3 - Divulgação / Comercialização,3,Música,33,Música Instrumental,51,Apresentação Musical,2010-05-06 10:49:13
3,132803,1012471,86608,Transporte Local / Locação de Automóvel / Comb...,134,Metro 3 / Km,4,500.0,4,1.45,...,MG,Viçosa,2 - Produção / Execução,7,Artes Integradas,71,Artes Integradas,51,Apresentação Musical,2010-12-21 17:09:22
4,115111,93932,11387,Confecção de painéis explicativos,90,Unidade,30,20.0,1,300.00,...,PE,Recife,2 - Produção / Execução,7,Artes Integradas,71,Artes Integradas,62,Exposição de Artes,2009-07-27 08:40:04


##### PLANILHA COMPROVACAO

QUERY:

    SELECT a.idPlanilhaAprovacao,c.idComprovantePagamento,a.IdPRONAC,
       a.idProduto as cdProduto,a.idEtapa as cdEtapa,a.idUFDespesa AS cdUF,a.idMunicipioDespesa as cdCidade,a.idPlanilhaItem,
	   c.nrComprovante,c.nrSerie,e.CNPJCPF as nrCNPJCPF,f.Descricao as nmFornecedor,
	   CASE c.tpDocumento
         WHEN 1 THEN ('Cupom Fiscal')
         WHEN 2 THEN ('Guia de Recolhimento')
         WHEN 3 THEN ('Nota Fiscal/Fatura')
         WHEN 4 THEN ('Recibo de Pagamento')
         WHEN 5 THEN ('RPA') ELSE ''
       END as tpDocumento,
       c.dtPagamento,
       dtEmissao,
       CASE
         WHEN c.tpFormaDePagamento = '1' THEN 'Cheque'
         WHEN c.tpFormaDePagamento = '2' THEN 'Transferencia Bancaria'
         WHEN c.tpFormaDePagamento = '3' THEN 'Saque/Dinheiro' ELSE ''
       END as tpFormaDePagamento,
       c.nrDocumentoDePagamento,c.idArquivo,c.dsJustificativa as dsJustificativaProponente,
	   b.dsJustificativa as dsOcorrenciaDoTecnico,b.stItemAvaliado,
	   CASE
	     WHEN stItemAvaliado = 1 THEN 'Validado'
	     WHEN stItemAvaliado = 3 THEN 'Impugnado'
	     WHEN stItemAvaliado = 4 THEN 'Aguardando analise'
	   END AS stAvaliacao,
	   c.vlComprovacao
      FROM       SAC.dbo.tbplanilhaaprovacao                                  a 
      INNER JOIN BDCORPORATIVO.scSAC.tbComprovantePagamentoxPlanilhaAprovacao b ON (a.idPlanilhaAprovacao    = b.idPlanilhaAprovacao)
      INNER JOIN BDCORPORATIVO.scSAC.tbComprovantePagamento                   c ON (b.idComprovantePagamento = c.idComprovantePagamento)
      INNER JOIN Agentes.dbo.Agentes                                          e ON (c.idFornecedor           = e.idAgente)
      INNER JOIN Agentes.dbo.Nomes                                            f ON (c.idFornecedor           = f.idAgente)
      WHERE    a.nrFonteRecurso = 109 
          AND (sac.dbo.fnVlComprovado_Fonte_Produto_Etapa_Local_Item
	      (a.idPronac,a.nrFonteRecurso,a.idProduto,a.idEtapa,a.idUFDespesa,a.idMunicipioDespesa, a.idPlanilhaItem)) > 0 
;


In [15]:
PlanilhaComprovacao = 'planilha_comprovacao_2.csv'
planilhaComprovacao = read_csv(PlanilhaComprovacao)

planilhaComprovacao.head()

,idPlanilhaAprovacao,idComprovantePagamento,IdPRONAC,cdProduto,cdEtapa,cdUF,cdCidade,idPlanilhaItem,nrComprovante,nrSerie,...,dtPagamento,dtEmissao,tpFormaDePagamento,nrDocumentoDePagamento,idArquivo,dsJustificativaProponente,dsOcorrenciaDoTecnico,stItemAvaliado,stAvaliacao,vlComprovacao
0,18552,4326,132451,1,1,35,355030,43,3052,nfse,...,2014-01-16 00:00:00,2013-03-27 00:00:00,Transferencia Bancaria,42501,410880,Elaboração de projeto gráfico do livro.,NaN,4,Aguardando analise,12000.0
1,18553,4332,132451,1,2,35,355030,91,287,nfse,...,2014-01-16 00:00:00,2012-04-30 00:00:00,Transferencia Bancaria,2945,410935,Reportagem e produção de textos para livro.,NaN,4,Aguardando analise,6000.0
2,18554,4755,132451,1,2,35,355030,143,130,nfse,...,2014-01-28 00:00:00,2012-03-16 00:00:00,Transferencia Bancaria,31601,420731,Fotografia artística para o livro.,NaN,4,Aguardando analise,12000.0
3,18555,4334,132451,1,2,35,355030,3204,139,nfse,...,2014-01-16 00:00:00,2013-08-27 00:00:00,Transferencia Bancaria,90201,410939,Tratamento de imagens para o livro.,NaN,4,Aguardando analise,6000.0
4,18556,4327,132451,1,2,35,355030,97,293,nfse,...,2014-01-16 00:00:00,2012-05-04 00:00:00,Transferencia Bancaria,2945,410917,Edição dos textos do livro.,NaN,4,Aguardando analise,8000.0


#### PLANILHA CAPTACAO
QUERY:

    SELECT Idcaptacao, capt.AnoProjeto+capt.Sequencial AS Pronac, NumeroRecibo, CgcCpfMecena, TipoApoio, MedidaProvisoria, DtChegadaRecibo, DtRecibo, CaptacaoReal, CaptacaoUfir, logon,isBemServico
    FROM SAC.dbo.Captacao capt

In [ ]:
PlanilhaCaptacao = r'planilha_captacao.csv'
planilhaCaptacao = pd.read_csv(PlanilhaCaptacao)

planilhaCaptacao.head()

### FEATURES )
    - Numero de Itens do projeto.
    - Total Comprovado.
    - Total Aprovado.
    - Total Captado.
    - Número de Comprovantes.
    - Numero de fornecedores de cada projeto. (% de fornecedores entre os mais comuns.)


In [ ]:
class Receipts:
    def __init__(self, raw_receipts):
        self.all_receipts = raw_receipts     
        
        
    def get_number_of_receipts(self):
        return len(self.all_receipts)

    def get_total_verified_cost(self):
        return sum(self.all_receipts['vlComprovacao'].values)
    
    def get_fornecedores(self):
        suppliers = self.all_receipts.nrCNPJCPF.unique()
        return suppliers


In [ ]:
class Captacoes:
    def __init__(self, raw_captacao):
        self.all_captacoes = raw_captacao     
        
    def get_total_real_raised_funds(self):
        return sum(self.all_captacoes['CaptacaoReal'].values) # "CaptacaoReal","CaptacaoUfir"
    

In [ ]:
class Items:
    def __init__(self, raw_items):
        self.all_items = raw_items
        
    
    def get_number_of_items(self):
        return self.all_items.shape[0]

    def get_quantity_of_items(self):
        quantity_of_items =  sum(self.all_items.QtItem * self.all_items.nrOcorrencia)
        return quantity_of_items
    
    def get_total_approved_cost(self):
        total_cost = sum(self.all_items.VlTotalAprovado)
        return total_cost
    
    def planilha_aprovacao_list(self):
        id_planilha_aprovacao = self.all_items['idPlanilhaAprovacao']
        return id_planilha_aprovacao.values

In [ ]:
class Project:
    def __init__(self, raw_project, raw_receipts, raw_captacoes):
        self.raw_project      = raw_project
        self.raw_receipts = raw_receipts
        self.raw_captacoes    = raw_captacoes
        
        self.pronac = self.setPronac()
        
        self.items = Items(self.setItemsList())
        self.receipts = Receipts(self.raw_receipts)
        self.captacoes = Captacoes(self.raw_captacoes)
        
        self.segment = self.setSegment()
        self.area = self.setArea()
        self.project_data = self.setProjetoData()
        
    def setItemsList(self):
        return self.raw_project.drop(['idPronac', 'PRONAC', 'Area', 'idArea','Segmento', 'idSegmento', 'DataProjeto'], axis=1)
    
    def setSegment(self):
        return [self.raw_project['Segmento'].iloc[0],self.raw_project['idSegmento'].iloc[0]]

    def setArea(self):
        return [self.raw_project['Area'].iloc[0],self.raw_project['idArea'].iloc[0]]
      
    def setPronac(self):
        return [self.raw_project['PRONAC'].iloc[0], self.raw_project['idPronac'].iloc[0]]

    def setProjetoData(self):
        return self.raw_project['DataProjeto'].iloc[0]
        
    

In [ ]:
class ProjectsList:
    
    def __init__(self, planilhaOrcamentaria, planilhaComprovacao, planilhaCaptacao):
        self.planilhaOrcamentaria = planilhaOrcamentaria
        self.planilhaComprovacao = planilhaComprovacao
        self.planilhaCaptacao = planilhaCaptacao
        
        self.pronac_list = []
        # self.loaded_projects_list = []
        self.loaded_projects = {}
        

    # CREATE LIST OF ALL PRONAC's
    def createPronacList(self):
        return self.planilhaOrcamentaria.PRONAC.unique()
    
    # Array of tuples: [(filter1,value1),(filter2,value2)]
    def createPronacListForFilteredProjects(self, filters_and_values):
        filtered_pronacs = []
        for (filtered_column, filter_value) in filters_and_values:
            filtered_projects = self.planilhaOrcamentaria[self.planilhaOrcamentaria[filtered_column] == filter_value]
            if (len(filtered_projects) > 0):
                filtered_pronacs = filtered_projects.PRONAC.unique()
        return filtered_pronacs
    
    # GET SINGLE PROJECT BY PRONAC
    def getUnloadedProject(self, pronac):
        raw_project = self.planilhaOrcamentaria.loc[self.planilhaOrcamentaria['PRONAC'] == pronac]
        raw_receipts = self.planilhaComprovacao.loc[self.planilhaComprovacao['IdPRONAC'] == raw_project['idPronac'].iloc[0]]
        raw_captacoes = self.planilhaCaptacao.loc[self.planilhaCaptacao['Pronac'] == pronac]
        project = Project(raw_project, raw_receipts, raw_captacoes)
        return project

    
    # LOAD ALL PROJECTS
    def loadAllProjects(self):
        pronac_list = self.createPronacList()
        self.pronac_list = []
        
        for [i,pronac] in enumerate(pronac_list):
            # print('loading: ', i, ' / ', len(self.pronac_list))
            #self.loaded_projects_list.append(pronac)
            self.loaded_projects[pronac] = self.getUnloadedProject(pronac);
            self.pronac_list.append(pronac)
            
            # AVOID LOADING ALL PROJECTS.
            if i == 1000:
                break;

    # LOAD ALL PROJECTS
    def loadFilteredProjects(self, filters_and_values):
        pronac_list = self.createPronacListForFilteredProjects(filters_and_values)
        self.pronac_list = []
        
        for [i,pronac] in enumerate(pronac_list):
            # print('loading: ', i, ' / ', len(self.pronac_list))
            #self.loaded_projects_list.append(pronac)

            self.loaded_projects[pronac] = self.getUnloadedProject(pronac)
            self.pronac_list.append(pronac)
            
            # AVOID LOADING ALL PROJECTS.
            if i == 1000:
                break;

    # GET SINGLE PROJECT BY PRONAC
    def loadSingleProject(self, pronac):
        self.pronac_list = [pronac]
        #self.loaded_projects_list = [self.loaded_projects[pronac]]
        self.loaded_projects[pronac] = self.getUnloadedProject(pronac)
        return self.loaded_projects[pronac]



In [ ]:
# projectList = ProjectsList(planilhaOrcamentaria, planilhaComprovacao, planilhaCaptacao)
# projectList.loadAllProjects()

#projectList.loadFilteredProjects([('PRONAC', 111986)])
#projectList.loaded_projects[projectList.pronac_list[0]].items.all_items

In [ ]:
# allprojects = projectList.loaded_pronacs

# for project in allprojects:
#     projectTest = projectList.getLoadedProject(project)
#     print(projectTest.pronac)
    

proj = ProjectsList(planilhaOrcamentaria, planilhaComprovacao, planilhaCaptacao).loadSingleProject(172085)

# ************** PROJETO **************
# proj.area
# proj.project_data
# proj.segment
# proj.pronac

# ************** COMPROVANTES **************
# proj.receipts.all_receipts
# proj.receipts.get_number_of_receipts()
# proj.receipts.get_total_verified_cost()
# proj.receipts.get_fornecedores()

# ************** ITEMS **************
# proj.items.all_items
# proj.items.get_number_of_items()
# proj.items.get_quantity_of_items()
# proj.items.get_total_approved_cost()

# ************** CAPTACAO **************
# proj.captacoes.get_total_real_raised_funds()

### Filter projects

In [ ]:
# projectList = ProjectsList(planilhaOrcamentaria, planilhaComprovacao, planilhaCaptacao)


# proj = projectList.loadSingleProject(172085)

# segmento_projeto = proj.segment[1]
# print('Segmento: ', segmento_projeto)

projectList = ProjectsList(planilhaOrcamentaria, planilhaComprovacao, planilhaCaptacao)

projectList.loadFilteredProjects([('idSegmento', '11')])
pronacs = projectList.pronac_list

# print(len(pronacs))
# print(pronacs)
# print('Itens:')
# for [i,pronac] in enumerate(pronacs):
#     print(i,')')
#     num_of_items = projectList.loaded_projects[pronac].items.get_number_of_items()
#     print(num_of_items)
    
a = [projectList.loaded_projects[pronac].items.get_number_of_items() for pronac in pronacs]


In [ ]:
import scipy
import matplotlib.pyplot as plt


plt.hist(a,bins=30)

# mean = np.mean(a)
# var  = np.var(a)

# print(mean)
# print(var)

# scipy.stats.norm(mean, var**.5).cdf(mean + 1.5*var**.5)



# norm(mean=mean, std=var)

In [ ]:
import scipy
outlier_probability_function = lambda mean, var: scipy.stats.norm(mean, var**.5).cdf(mean + 1.5*var**.5)

In [ ]:
def is_gaussian_outlier(x, mean, standard_deviation, c = 1.5):
    return x > mean + c * standard_deviation

def is_project_num_items_outlier(pronac):
    projectList = ProjectsList(planilhaOrcamentaria, planilhaComprovacao, planilhaCaptacao)

    project = projectList.loadSingleProject(pronac)
    segment, segment_id = project.segment
    project_num_items = project.items.get_number_of_items()
    print(project_num_items)
    
    projectList.loadFilteredProjects([('idSegmento', segment_id)])
    pronacs = projectList.pronac_list

    projects_number_of_items = [projectList.loaded_projects[pronac].items.get_number_of_items() for pronac in pronacs]
    
    mean = np.mean(projects_number_of_items)
    #variance = np.var(projects_number_of_items)
    std = np.std(projects_number_of_items)
    
    is_outlier = is_gaussian_outlier(project_num_items, mean, std,1.5)
    return is_outlier

pronac = 172085

p = is_project_num_items_outlier(pronac)
print('local outlier probability = {}'.format(p))

In [ ]:
pronac = 172085

p = local_outlier_probability(pronac)
# proj = projectList.loadSingleProject(172085)



In [ ]:
import scipy
p = lambda mean, var: scipy.stats.norm(mean, var**.5).cdf(mean + 1.5*var**.5)

print(p(0, 1))
print(p(10, 654))
print(p(213, 23))

In [ ]:
projectList = ProjectsList(planilhaOrcamentaria, planilhaComprovacao, planilhaCaptacao)


print(projectList.loadSingleProject(172207).pronac)

### Most common items of a given segment

### Most common suppliers of a given segment/state

### Generating Features Dataset